# Notebook for exporting data collected at SSRL 1-5

In [20]:
# imports, setup
from databroker import Broker
#db = Broker.named('mongoCat')
import os
import suitcase.csv
import suitcase.tiff_series
import suitcase.json_metadata

from itertools import tee

def std_exporter(docs, directory, fn, overwrite=False):
    '''
    Batch exporter for data.  

    docs: documents from a Bluesky Run.  

    directory: Parent directory where exported files will be written to.
                Files will be further separated into subdirectories
    '''

    docs1, docs2, docs3 = tee(docs, 3)
    suitcase.csv.export(docs1, directory, f'scan/{fn}-')
    suitcase.tiff_series.export(docs2, directory, f'tiff/{fn}-')
    suitcase.json_metadata.export(docs3, directory, f'meta/{fn}-')


In [21]:
# dummy data
db = Broker.named('temp')
from bluesky import RunEngine
RE = RunEngine()
RE.subscribe(db.insert)
from bluesky.plans import scan
from ophyd.sim import det, motor


RE(scan([det], motor, -1, 1, num=5))


(&#39;de24dffc-6d19-4452-be3d-e86932eacfc6&#39;,)

In [22]:
# check connections
## see if we can see data
db[-1].table(fill=True)


,time,det,motor,motor_setpoint
seq_num,,,,
1,2020-10-01 17:57:14.204423666,0.606531,-1.0,-1.0
2,2020-10-01 17:57:14.208385706,0.882497,-0.5,-0.5
3,2020-10-01 17:57:14.212374210,1.000000,0.0,0.0
4,2020-10-01 17:57:14.216364145,0.882497,0.5,0.5
5,2020-10-01 17:57:14.219407558,0.606531,1.0,1.0


In [26]:
# User Settings

filename = 'testName_'
user_name = 'b_mehta'
export_path = './testexport/'

START = 1 #  scan id
STOP = 1

In [27]:
hdr = db[START]
hdr.start

{&#39;uid&#39;: &#39;de24dffc-6d19-4452-be3d-e86932eacfc6&#39;,
 &#39;time&#39;: 1601575034.1864767,
 &#39;versions&#39;: {&#39;ophyd&#39;: &#39;1.5.0&#39;, &#39;bluesky&#39;: &#39;1.6.2&#39;},
 &#39;scan_id&#39;: 1,
 &#39;plan_type&#39;: &#39;generator&#39;,
 &#39;plan_name&#39;: &#39;scan&#39;,
 &#39;detectors&#39;: [&#39;det&#39;],
 &#39;motors&#39;: [&#39;motor&#39;],
 &#39;num_points&#39;: 5,
 &#39;num_intervals&#39;: 4,
 &#39;plan_args&#39;: {&#39;detectors&#39;: [&quot;SynGauss(prefix=&#39;&#39;, name=&#39;det&#39;, read_attrs=[&#39;val&#39;], configuration_attrs=[&#39;Imax&#39;, &#39;center&#39;, &#39;sigma&#39;, &#39;noise&#39;, &#39;noise_multiplier&#39;])&quot;],
  &#39;num&#39;: 5,
  &#39;args&#39;: [&quot;SynAxis(prefix=&#39;&#39;, name=&#39;motor&#39;, read_attrs=[&#39;readback&#39;, &#39;setpoint&#39;], configuration_attrs=[&#39;velocity&#39;, &#39;acceleration&#39;])&quot;,
   -1,
   1],
  &#39;per_step&#39;: &#39;None&#39;},
 &#39;hints&#39;: {&#39;dimensions&#39;: [[[&#39;motor&#39;], &#39;primary&#39;]]},
 &#39;plan_pattern&#39;: &#39;inner_product&#39;,
 &#39;plan_pattern_module&#39;: &#39;bluesky.plan_patterns&#39;,
 &#39;plan_pattern_args&#39;: {&#39;num&#39;: 5,
  &#39;args&#39;: [&quot;SynAxis(prefix=&#39;&#39;, name=&#39;motor&#39;, read_attrs=[&#39;readback&#39;, &#39;setpoint&#39;], configuration_attrs=[&#39;velocity&#39;, &#39;acceleration&#39;])&quot;,
   -1,
   1]}}

In [28]:
# Export Loop
for scan_id in range(START, STOP + 1):
    h = db[scan_id]
    fn = f'{filename + str(scan_id)}'

    try:
        std_exporter(h.documents(), export_path, fn)
        print('expoted scan #{scan_id}')
    except Exception as e:
        print(f'export failed')
        print(e)

